## Step 1: Installing libraries

You only need to install required libraries when you're running the notebooks locally (e.g. on your own computer).

The only _required_ library is the toolkit library, which you'll use to write your formula. You can install it by running the following command in your terminal:

```bash
pip install pybox-toolkit
```

You can also install the library from within the Jupyter notebook, by adding the following line into a code cell:

```
%pip install pybox-toolkit
```

Please only install the libraries on the list below, as they are guaranteed to be present within the website environment as well. Any other python packages, apart from the standard library, may not be available when running in the website.

### List of available libraries

 - [`sympy`](https://www.sympy.org/en/index.html)

## Step 2: Importing libraries

Import required libraries. At the minimum, you need to import the `toolkit` library, but you can also rely on `sympy` or the python standard library.

The `toolkit` library acts as the interface between your Jupyter notebook and the website. In addition, it also provides you with some convenient features, such as physical bounds checking, SI units, unit composition, etc, but these features will be discussed later as we encounter them.

In [1]:
# Required import for any formula
import toolkit
# Importing the Python math library (https://docs.python.org/3/library/math.html)
# We will use the library to obtain constants, such as `math.pi` and `math.e`
import math
# Inside of pure formulas, we need to use the math functions provided by sympy
# This is explained later on
from sympy import sin, log, exp
from scipy.stats import gumbel_r

## Step 3: Using units

As the library is designed with engineering formulas in mind, it has support for units. You have three different options:

 - stick to the SI units provided by the toolkit library
 - create your own units
 - create new units by composition of existing units

### Using provided SI units 

Available units in the library are:

 - Meter
 - Second
 - Kilogram
 - Ampere
 - Kelvin
 - Mole
 - Candela

You can import them from the toolkit library using a simple import statement

In [2]:
from toolkit.typing import Unit

class Probability(Unit):
    """Probability"""
    units = "[-]"
    physical_range = "[0, 1]"

class RandomVariable(Unit):
    """Random variable with unknown units"""
    units = "[-]"
    physical_range = "(-inf, inf)"

class ParameterMu(Unit):
    """Distribution parameter mu with unknown units"""
    units = "[-]"
    physical_range = "(-inf, inf)"

class ParameterBeta(Unit):
    """Distribution parameter beta with unknown units"""
    units = "[-]"
    physical_range = "[0, inf)"

In [10]:
@toolkit.ImpureFormula(
    outputs = [("mu", ParameterMu("Parameter mu")),
               ("beta", ParameterBeta("Parameter beta")),
               ("p", Probability("Probability from the Gumbel CDF, F(x)")),
               ("exceedance", Probability("Probability from the Gumbel CDF, F(x)"))],
    keywords = ["Probability", "Gumbel", "Distribution Parameters", "CDF", "scipy"]
)
def gumbel_2_points_scipy(p_1: Probability("Probability at Point 1"),
                    x_1: RandomVariable("Random variable at Point 1"),
                    p_2: Probability("Probability at Point 2"),
                    x_2: RandomVariable("Random variable at Point 2"),
                    x: RandomVariable("Random variable")):
    """Compute Gumbel distribution parameters from two points of the CDF."""
    
    beta = -(x_2 - x_1) / log(log(p_2) / log(p_1))
    mu = x_1 + beta * log(-log(p_1))
    
    gumbel = gumbel_r(loc=mu, scale=beta)

    p = gumbel.cdf(x)
    exceedance = 1 - p

    return mu, beta, p, exceedance

In [11]:
gumbel_2_points_scipy(0.9889524805037951, 5,
                0.9984976904055608, 7,
                6)

[{'mu': 0.500000000000059,
  'beta': 0.999999999999987,
  'p': 0.9959215680475387,
  'exceedance': 0.00407843195246127}]

# Documentation: gumbel_2_points_scipy

Given two sets of points from the CDF, find the parameters of the Gumbel distribution, $\mu$ and $\beta$.

The distribution of random variable $x$ is defined by the CDF:

$$
F-x(x) = \exp(-\exp(-\frac{x-u}{\beta}))
$$

and the points for fitting the parameters are:

$$
\{p_i,x_i\}:\quad F(x_i)=p_i
$$

for $i=1,2$.

In [16]:
@toolkit.test.set_function(
    gumbel_2_points_scipy
)
class test_gumbel_2_points_scipy(toolkit.test.ToolkitTests):
    @toolkit.test.test
    def set_1(self):
        self.documented_test(
            arguments = {"p_1": 0.9889524805037951,
                         "x_1": 5.0,
                         "p_2": 0.9984976904055608,
                         "x_2": 7.0,
                         "x": 6.0}, 
            expected = {"mu": 0.5,
                        "beta": 1.0,
                        "p": 0.995921568047539,
                        "exceedance": 0.00407843195246127}
        )
    
    # @toolkit.test.test
    # def radius2(self):
    #     calculated_radius = self.function(area = 4)[0]["radius"] # Outputs follow the form: [{"radius": ...}, {"radius": ...}, ...]
    #     self.assertTrue(calculated_radius < 2) # This won't be documented, but will run

In [17]:
gumbel_2_points_scipy.run_tests()

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


True